# 회귀모델에 Bert_time을 이용해 가능한 CI_HOUR 리스트를 만든 파일 적용하기

## 가능한 CI_HOUR 리스트에서 회귀모델과 가장 가까운 값을 회귀모델과 비교하여 최종 CI_HOUR 1개로 결정한다

In [7]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from tqdm import tqdm

In [14]:
a = pd.read_csv('./out_csvs/use_bertTime_and_get_CI_HOURS.csv')
b = pd.read_csv('./out_csvs/auto6.csv')

# 문자열 형태의 리스트를 파이썬 리스트로 변환
def convert_str_to_list(value):
    try:
        return eval(value)
    except:
        return [value]

a['CI_HOUR'] = a['CI_HOUR'].apply(convert_str_to_list)

for i in tqdm(range(len(b))):
    current_a_value = a['CI_HOUR'].iloc[i]
    current_b_value = b['CI_HOUR'].iloc[i]
    
    # a의 값이 -1일 경우 b의 값을 사용
    if current_a_value == [-1]:
        a['CI_HOUR'].iloc[i] = current_b_value
    # a의 값이 리스트일 경우 b의 값과 가장 가까운 값을 선택
    elif isinstance(current_a_value, list):
        closest_value = min(current_a_value, key=lambda x: abs(x - current_b_value))
        if abs(closest_value - current_b_value) > 71 * current_b_value: # 차이가 너무 클 경우 회귀모델 예측값 적용
            a['CI_HOUR'].iloc[i] = current_b_value
        elif closest_value > 2200: # cloest value가 2200을 넘을 경우 회귀모델 예측값을 적용
            a['CI_HOUR'].iloc[i] = current_b_value
        else:
            a['CI_HOUR'].iloc[i] = closest_value # 나머지는 cloest value 적용

# 최종결과를 저장
a.to_csv('./out_csvs/final_auto06.csv', index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 220491/220491 [01:23<00:00, 2656.26it/s]
